In [ ]:
import pandas as pd
import numpy as np
import scipy
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scripts.python.routines.betas import betas_drop_na
from plotly.subplots import make_subplots
from scipy import stats
import pickle
import random
import plotly.express as px
import copy
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scripts.python.pheno.datasets.filter import filter_pheno
from scripts.python.pheno.datasets.features import get_column_name, get_status_dict, get_sex_dict
from scripts.python.routines.plot.scatter import add_scatter_trace
import plotly.graph_objects as go
import pathlib
from scripts.python.routines.manifest import get_manifest
from scripts.python.routines.plot.save import save_figure
from scripts.python.routines.plot.layout import add_layout, get_axis
from scripts.python.routines.plot.p_value import add_p_value_annotation
from statsmodels.stats.multitest import multipletests
from sklearn.metrics import mean_absolute_error
from impyute.imputation.cs import fast_knn, mean, median, random, mice, mode, em
import plotly.io as pio
pio.kaleido.scope.mathjax = None
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=False)
from functools import reduce
from scipy.stats import kruskal, mannwhitneyu

# Init data

In [ ]:
dataset = "GSEUNN"
path = f"E:/YandexDisk/Work/pydnameth/datasets"
datasets_info = pd.read_excel(f"{path}/datasets.xlsx", index_col='dataset')
platform = datasets_info.loc[dataset, 'platform']
manifest = get_manifest(platform, path=path)

path_save = f"{path}/{platform}/{dataset}/special/030_immuno_generate_subset"
pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)

df_immuno_genes = pd.read_excel(f"{path}/{platform}/{dataset}/data/immuno/immuno_markers_genes.xlsx")
dict_immuno_genes = dict(zip(df_immuno_genes['immuno_marker'], df_immuno_genes['gene']))
feats_immuno = df_immuno_genes['immuno_marker']
feats_gene = df_immuno_genes['gene']

df_all = pd.read_excel(f"{path}/{platform}/{dataset}/special/021_ml_data/immuno/345_imp(fast_knn)_replace(quarter).xlsx", index_col="index")

df_model_res = pd.read_excel(f"{path}/{platform}/{dataset}/special/021_ml_data/immuno/models/immuno_trn_val_lightgbm/multiruns/2022-07-19_23-23-45/10/predictions.xlsx", index_col="index")

df_all = pd.merge(df_all, df_model_res.loc[:, 'Estimation'], left_index=True, right_index=True)
df_all['AbsAgeDiff'] = np.abs(df_all["Age"] - df_all["Estimation"])
df_all.to_excel(f"{path_save}/data.xlsx")